# ECS 171 Group Project - Group 10 - ???

In [1]:
!pip install blitz-bayesian-pytorch
!pip install ray
!pip install tensorboardX

     |████████████████████████████████| 51kB 4.7MB/s 
     |████████████████████████████████| 2.2MB 11.9MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


     |████████████████████████████████| 23.1MB 138kB/s 
     |████████████████████████████████| 133kB 54.6MB/s 
     |████████████████████████████████| 1.3MB 52.5MB/s 
     |████████████████████████████████| 71kB 10.4MB/s 
     |████████████████████████████████| 2.9MB 58.4MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
     |████████████████████████████████| 204kB 52.5MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 
     |████████████████████████████████| 143kB 54.1MB/s 
     |████████████████████████████████| 296kB 58.0MB/s 
     |████████████████████████████████| 61kB 9.8MB/s 
     |████████████████████████████████| 102kB 13.9MB/s 
  Created wheel for gpustat: filename=gpustat-0.6.0-cp36-none-any.whl size=12622 sha256=c364e105a2ca57cab772174cd6e76dff6a66a4f5153d1b136d2c1d29ec425486
  Stored in directory: /root/.cache/pip/wheels/48/b4/d5/fb5b7f1d040f2ff20687e3bad6867d63155dbde5a7c10f4293
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-cp36-none-any.whl s

In [1]:
#dont run this unless you are hosting data anad notebook in google drive
from google.colab import drive
drive.mount('/content/drive/')
import os
os.chdir('/content/drive/My Drive/171-g1/notebook') #change this path with folder containing your code

Mounted at /content/drive/


In [9]:
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
from blitz.modules import BayesianLinear
from blitz.utils import variational_estimator
from sklearn.preprocessing import StandardScaler
from blitz.modules.base_bayesian_module import BayesianModule
from blitz.modules.weight_sampler import TrainableRandomDistribution
from blitz.losses import kl_divergence_from_nn
from blitz.modules.base_bayesian_module import BayesianModule, BayesianRNN
from hyperopt import hp
from ray import tune
from ray.tune.suggest.hyperopt import HyperOptSearch
from ray.tune import CLIReporter
from ray.tune.schedulers import ASHAScheduler
np.random.seed(171)

In [3]:
# Code  source for BNN https://towardsdatascience.com/making-your-neural-network-say-i-dont-know-bayesian-nns-using-pyro-and-pytorch-b1c24e6ab8cd
def load_data(data_dir=os.path.abspath("/content/drive/My Drive/171-g1/Data/")):
    X_train = pd.read_csv(os.path.join(data_dir,"X_train.csv"), delimiter = ',', header = None)
    X_test = pd.read_csv(os.path.join(data_dir,"X_test.csv"), delimiter = ',', header = None)
    y_train = pd.read_csv(os.path.join(data_dir,"Y_train.csv"), header = None)
    y_test = pd.read_csv(os.path.join(data_dir,"Y_test.csv"), header = None)

    X_train, y_train = torch.tensor(np.array(X_train)).float(), torch.tensor(np.array(y_train)).float()
    X_test, y_test = torch.tensor(np.array(X_test)).float(), torch.tensor(np.array(y_test)).float()
    return X_train, X_test, y_train, y_test

In [4]:
#variational_estimator
class BayesianRegressor(nn.Module):
    def __init__(self, conf):
        super().__init__()
        #self.linear = nn.Linear(input_dim, output_dim)
        self.blinear1 = BayesianLinear(81 #terrible that this is hardcoded but im tryna seem sumn
                                       , conf['input_dim'])
        self.blinear2 = BayesianLinear(conf['input_dim'], conf['output_dim'])
        self.sample_nbr = conf['sample_nbr'] #TODO: same thing
        self.criterion = conf['criterion']
        self.complexity_cost_weight = conf['complexity_cost_weight']
        
    def forward(self, x):
        x_ = self.blinear1(x)
        return self.blinear2(x_)
    
    
#from https://github.com/piEsposito/blitz-bayesian-deep-learning/blob/3526b57058e0914105e17cf7e93682ef11a9cd28/blitz/utils/variational_estimator.py#L35    
    def nn_kl_divergence(self):
        """Returns the sum of the KL divergence of each of the BayesianModules of the model, which are from
            their posterior current distribution of weights relative to a scale-mixtured prior (and simpler) distribution of weights
            Parameters:
                N/a
            Returns torch.tensor with 0 dim.      
        
        """
        return kl_divergence_from_nn(self)
    
    def sample_elbo(self,
                    inputs,
                    labels,
                    #criterion,
                    #sample_nbr, #hyperparam
                    #complexity_cost_weight=1 #hyperparam
                    ):

        """ Samples the ELBO Loss for a batch of data, consisting of inputs and corresponding-by-index labels
                The ELBO Loss consists of the sum of the KL Divergence of the model 
                 (explained above, interpreted as a "complexity part" of the loss)
                 with the actual criterion - (loss function) of optimization of our model
                 (the performance part of the loss). 
                As we are using variational inference, it takes several (quantified by the parameter sample_nbr) Monte-Carlo
                 samples of the weights in order to gather a better approximation for the loss.
            Parameters:
                inputs: torch.tensor -> the input data to the model
                labels: torch.tensor -> label data for the performance-part of the loss calculation
                        The shape of the labels must match the label-parameter shape of the criterion (one hot encoded or as index, if needed)
                criterion: torch.nn.Module, custom criterion (loss) function, torch.nn.functional function -> criterion to gather
                            the performance cost for the model
                sample_nbr: int -> The number of times of the weight-sampling and predictions done in our Monte-Carlo approach to 
                            gather the loss to be .backwarded in the optimization of the model.        
        
        """

        loss = 0
        for _ in range(self.sample_nbr):
            outputs = self(inputs)
            loss += self.criterion(outputs, labels) 
            loss += self.nn_kl_divergence() * self.complexity_cost_weight
            
        return (loss / (self.sample_nbr))

In [18]:
def train_model(conf, checkpoint_dir="../checkpoint/", data_dir=os.path.abspath("/content/drive/My Drive/171-g1/Data/")):
    net = BayesianRegressor(conf)

    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
        if torch.cuda.device_count() > 1:
            net = nn.DataParallel(net)
    net.to(device)
    print("current device: " + device)

    optimizer = optim.Adam(net.parameters(), lr=conf['opt_lr'])
    criterion = conf['criterion']

    if checkpoint_dir:
        model_state, optimizer_state = torch.load(
            os.path.join(checkpoint_dir, "checkpoint"))
        net.load_state_dict(model_state)
        optimizer.load_state_dict(optimizer_state)

    X_train, X_test, y_train, y_test=load_data() 

    ds_train = torch.utils.data.TensorDataset(X_train, y_train)
    test_abs = int(len(ds_train) * 0.8)
    train_subset, val_subset = random_split(
        ds_train, [test_abs, len(ds_train) - test_abs])
    trainloader = torch.utils.data.DataLoader(train_subset,
                                                    batch_size=conf["batch_size"],
                                                    shuffle=True,
                                                    num_workers=0) 
    ds_test = torch.utils.data.TensorDataset(X_test, y_test)
    valloader = torch.utils.data.DataLoader(val_subset,
                                            batch_size=conf["batch_size"],
                                            shuffle=True,
                                            num_workers=0)

    for epoch in range(10):  # loop over the dataset multiple times
        running_loss = 0.0
        epoch_steps = 0
        for i, data in enumerate(trainloader, 0):
            # get the inputs; data is a list of [inputs, labels]
            inputs, labels = data
            inputs, labels = inputs.to(device), labels.to(device)

            # zero the parameter gradients
            optimizer.zero_grad()

            # forward + backward + optimize
            outputs = net(inputs)
            loss = net.sample_elbo(inputs=inputs,
                           labels=labels)
            loss.backward()
            optimizer.step()

            # print statistics
            running_loss += loss.item()
            epoch_steps += 1
            if i % 2000 == 1999:  # print every 2000 mini-batches
                print("[%d, %5d] loss: %.3f" % (epoch + 1, i + 1,
                                                running_loss / epoch_steps))
                running_loss = 0.0

        # Validation loss
        val_loss = 0.0
        val_steps = 0
        total = 0
        correct = 0
        for i, data in enumerate(valloader, 0):
            with torch.no_grad():
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)

                outputs = net(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

                loss = criterion(outputs, labels)
                val_loss += loss.cpu().numpy()
                val_steps += 1

        with tune.checkpoint_dir(epoch) as checkpoint_dir:
            path = os.path.join(checkpoint_dir, "checkpoint")
            torch.save((net.state_dict(), optimizer.state_dict()), path)

        tune.report(loss=(val_loss / val_steps), accuracy=correct / total)
    print("Finished Training")

In [6]:
## go through and adapt

def test_accuracy(net, device="cpu"):
    X_train, X_test, y_train, y_test=load_data() 

    ds_test = torch.utils.data.TensorDataset(X_test, y_test)
    testloader = torch.utils.data.DataLoader(
        ds_test, batch_size=4, shuffle=False, num_workers=2)

    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    return correct / total

In [7]:
from ray import tune
from ray.tune import CLIReporter
import torch.nn.functional as F
from functools import partial
from hyperopt import pyll
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample
import tensorboardX
from torch.utils.data import random_split

In [19]:
## go through and adapt https://colab.research.google.com/github/pytorch/tutorials/blob/gh-pages/_downloads/c24b93738bc036c1b66d0387555bf69a/hyperparameter_tuning_tutorial.ipynb#scrollTo=Hmn9JEYO9MJs
def main(num_samples=10, max_num_epochs=10, gpus_per_trial=1):
    data_dir = os.path.abspath("/content/drive/My Drive/171-g1/Data/")
    load_data(data_dir)
    conf = {
    'input_dim' : (2 ** scope.int(hp.quniform('input_dim', 2, 9, 1))),
    'output_dim' : 1, #(2 ** scope.int(hp.quniform('output_dim', 2, 9, 1))),
    'criterion' : torch.nn.MSELoss(),
    'epoch' : 10,
    'eval_samples' : 100,
    'eval_std_multiplier' : 3,
    'opt_lr' : hp.uniform('opt_lr', 1e-10, 0.1),
    'sample_nbr' : 32,
    'std_multiplier' : 3, #hparam(?)
    'batch_size' : hp.choice('batch_size', [2, 4, 8, 16]),
    'complexity_cost_weight' : 1
    }

    hyperopt_search = HyperOptSearch(conf, max_concurrent=2) # probably dont need this tbh

    scheduler = ASHAScheduler(
        metric="loss",
        mode="min",
        max_t=max_num_epochs,
        grace_period=1,
        reduction_factor=2)
    reporter = CLIReporter(
        #parameter_columns=["input_dim", "output_dim", "opt_lr", "batch_size"],
        metric_columns=["loss", "accuracy", "training_iteration"])
    result = tune.run(
        partial(train_model, data_dir=data_dir),
        resources_per_trial={"cpu": 2, "gpu": gpus_per_trial},
        config=conf,
        num_samples=num_samples,
        search_alg=hyperopt_search,
        scheduler=scheduler,
        progress_reporter=reporter)

    best_trial = result.get_best_trial("loss", "min", "last")
    print("Best trial config: {}".format(best_trial.config))
    print("Best trial final validation loss: {}".format(
        best_trial.last_result["loss"]))
    print("Best trial final validation accuracy: {}".format(
        best_trial.last_result["accuracy"]))

    best_trained_model = BayesianRegressor(best_trial.config["input_dim"], best_trial.config["output_dim"])
    device = "cpu"
    if torch.cuda.is_available():
        device = "cuda"
        if gpus_per_trial > 1:
            best_trained_model = nn.DataParallel(best_trained_model)
    best_trained_model.to(device)

    best_checkpoint_dir = best_trial.checkpoint.value
    model_state, optimizer_state = torch.load(os.path.join(
        best_checkpoint_dir, "checkpoint"))
    best_trained_model.load_state_dict(model_state)


    test_acc = test_accuracy(best_trained_model, device)
    print("Best trial test set accuracy: {}".format(test_acc))


if __name__ == "__main__":
    # You can change the number of GPUs per trial here:
    main(num_samples=10, max_num_epochs=10, gpus_per_trial=1)

2020-12-01 06:16:05,223	WARNING suggestion.py:70 -- DeprecationWarning: `max_concurrent` is deprecated for this search algorithm. Use tune.suggest.ConcurrencyLimiter() instead. This will raise an error in future versions of Ray.
2020-12-01 06:16:05,233	WARNING experiment.py:274 -- No name detected on trainable. Using DEFAULT.
2020-12-01 06:16:05,234	INFO registry.py:65 -- Detected unknown callable for trainable. Converting to class.


== Status ==
Memory usage on this node: 3.7/12.7 GiB
Using AsyncHyperBand: num_stopped=0
Bracket: Iter 8.000: None | Iter 4.000: None | Iter 2.000: None | Iter 1.000: None
Resources requested: 2/2 CPUs, 1/1 GPUs, 0.0/7.18 GiB heap, 0.0/2.44 GiB objects (0/1.0 accelerator_type:T4)
Result logdir: /root/ray_results/DEFAULT_2020-12-01_06-16-05
Number of trials: 1/10 (1 RUNNING)
+------------------+----------+-------+--------------+--------------------------+-------------+---------+----------------+-----------------------+-------------+-----------+--------------+--------------+------------------+
| Trial name       | status   | loc   |   batch_size |   complexity_cost_weight | criterion   |   epoch |   eval_samples |   eval_std_multiplier |   input_dim |    opt_lr |   output_dim |   sample_nbr |   std_multiplier |
|------------------+----------+-------+--------------+--------------------------+-------------+---------+----------------+-----------------------+-------------+-----------+-------

KeyboardInterrupt: ignored